# 中国宏观经济分析报告

本报告旨在通过对中国近30年GDP总量、CPI和PMI等关键宏观经济指标的分析，判断当前中国经济所处的阶段，并针对不同主体给出策略建议。报告内容包括数据获取、清洗处理、可视化、经济周期分析和策略建议。

## 1. 数据获取

我们使用`akshare`库从权威数据源获取中国宏观经济数据，包括年度GDP总量、CPI同比数据和制造业PMI数据。

In [ ]:
import akshare as ak
import pandas as pd

def get_gdp_data():
    gdp_yearly = ak.macro_china_gdp_yearly()
    gdp_yearly.rename(columns={
        '年份': '日期',
        '今值': 'GDP_亿元',
        '第一产业': '第一产业_亿元',
        '第二产业': '第二产业_亿元',
        '第三产业': '第三产业_亿元'
    }, inplace=True)
    gdp_yearly['日期'] = pd.to_datetime(gdp_yearly['日期'], format='%Y')
    gdp_yearly['日期'] = gdp_yearly['日期'].dt.to_period('Y')
    return gdp_yearly

def get_cpi_data():
    cpi_yearly = ak.macro_china_cpi_yearly()
    cpi_yearly.rename(columns={
        '月份': '日期',
        '今值': 'CPI_同比'
    }, inplace=True)
    cpi_yearly['日期'] = pd.to_datetime(cpi_yearly['日期'], format='%Y%m')
    cpi_yearly['日期'] = cpi_yearly['日期'].dt.to_period('M')
    return cpi_yearly

def get_pmi_data():
    pmi_data = ak.macro_china_pmi_yearly()
    pmi_data.rename(columns={
        '月份': '日期',
        '今值': 'PMI'
    }, inplace=True)
    pmi_data['日期'] = pd.to_datetime(pmi_data['日期'], format='%Y%m')
    pmi_data['日期'] = pmi_data['日期'].dt.to_period('M')
    return pmi_data

gdp_yearly_df = get_gdp_data()
cpi_df = get_cpi_data()
pmi_df = get_pmi_data()

gdp_yearly_df.to_csv('../data/gdp_yearly.csv', index=False)
cpi_df.to_csv('../data/cpi.csv', index=False)
pmi_df.to_csv('../data/pmi.csv', index=False)

print('数据获取完成并保存到CSV文件。')

数据获取完成并保存到CSV文件。


## 2. 数据清洗与处理

对获取到的数据进行同比计算，并尝试检测趋势拐点。由于"三驾马车"数据获取困难，暂时跳过其结构占比计算。

In [ ]:
import pandas as pd

def calculate_growth_rates(df, value_col, date_col):
    df = df.sort_values(by=date_col)
    df[f'{value_col}_同比'] = df[value_col].pct_change(periods=1) * 100
    return df

def detect_inflection_points(series):
    diff = series.diff().dropna()
    inflection_points = []
    for i in range(1, len(diff) - 1):
        if (diff.iloc[i] > 0 and diff.iloc[i-1] < 0 and diff.iloc[i+1] > 0) or \
           (diff.iloc[i] < 0 and diff.iloc[i-1] > 0 and diff.iloc[i+1] < 0):
            inflection_points.append(series.index[i+1])
    return inflection_points

gdp_yearly_df = pd.read_csv('../data/gdp_yearly.csv')
cpi_df = pd.read_csv('../data/cpi.csv')
pmi_df = pd.read_csv('../data/pmi.csv')

gdp_yearly_df['日期'] = pd.PeriodIndex(gdp_yearly_df['日期'], freq='Y')
cpi_df['日期'] = pd.PeriodIndex(cpi_df['日期'], freq='M')
pmi_df['日期'] = pd.PeriodIndex(pmi_df['日期'], freq='M')

gdp_yearly_df = calculate_growth_rates(gdp_yearly_df, 'GDP_亿元', '日期')
cpi_df = calculate_growth_rates(cpi_df, 'CPI_同比', '日期')
pmi_df = calculate_growth_rates(pmi_df, 'PMI', '日期')

gdp_inflection_points = detect_inflection_points(gdp_yearly_df['GDP_亿元'])
cpi_inflection_points = detect_inflection_points(cpi_df['CPI_同比'])
pmi_inflection_points = detect_inflection_points(pmi_df['PMI'])

gdp_yearly_df.to_csv('../data/gdp_yearly_processed.csv', index=False)
cpi_df.to_csv('../data/cpi_processed.csv', index=False)
pmi_df.to_csv('../data/pmi_processed.csv', index=False)

print('数据处理完成。')
print('GDP 拐点: ', gdp_inflection_points)
print('CPI 拐点: ', cpi_inflection_points)
print('PMI 拐点: ', pmi_inflection_points)

数据处理完成。
GDP 拐点:  [9, 12, 38, 42, 47, 50, 52]
CPI 拐点:  [13, 20, 22, 38, 50, 60, 64, 71, 77, 93, 98, 101, 106, 121, 123, 132, 147, 156, 166, 168, 170, 172, 180, 182, 187, 194, 196, 202, 206, 208, 210, 218, 230, 235, 237, 241, 243, 247, 250, 253, 256, 261, 264, 268, 281, 283, 291, 294, 304, 307, 315, 320, 322, 332, 334, 353, 356, 358, 368, 374, 386, 398, 409, 413, 415, 421, 425, 429, 431, 441, 443, 445, 449, 452, 462, 464]
PMI 拐点:  [3, 7, 16, 19, 26, 28, 31, 35, 37, 40, 45, 47, 53, 60, 62, 64, 67, 71, 75, 79, 81, 83, 88, 92, 102, 110, 121, 132, 143, 145, 151, 155, 161, 164, 171, 183, 185, 191, 195, 202, 204, 207, 210, 212, 214, 216, 219, 223, 229, 234, 237, 239, 242]


## 3. 数据可视化

我们将GDP、CPI和PMI数据通过折线图进行可视化，以便直观地观察其变化趋势。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams["font.sans-serif"] = ["WenQuanYi Zen Hei"]
plt.rcParams["axes.unicode_minus"] = False

colors = ["#4080FF", "#57A9FB", "#37D4CF", "#23C343", "#FBE842", "#FF9A2E", "#A9AEB8"]

def plot_gdp(df):
    plt.figure(figsize=(12, 6))
    plt.plot(df["日期"].astype(str), df["GDP_亿元"], marker="o", color=colors[0], label="GDP（亿元）")
    plt.title("中国年度GDP总量变化趋势")
    plt.xlabel("年份")
    plt.ylabel("GDP（亿元）")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.savefig("../images/gdp_trend.png")
    plt.close()

def plot_cpi(df):
    plt.figure(figsize=(12, 6))
    plt.plot(df["日期"].astype(str), df["CPI_同比"], marker="o", color=colors[1], label="CPI同比（%）")
    plt.title("中国CPI同比变化趋势")
    plt.xlabel("月份")
    plt.ylabel("CPI同比（%）")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.savefig("../images/cpi_trend.png")
    plt.close()

def plot_pmi(df):
    plt.figure(figsize=(12, 6))
    plt.plot(df["日期"].astype(str), df["PMI"], marker="o", color=colors[2], label="PMI")
    plt.title("中国制造业PMI变化趋势")
    plt.xlabel("月份")
    plt.ylabel("PMI")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.savefig("../images/pmi_trend.png")
    plt.close()

gdp_df = pd.read_csv("../data/gdp_yearly_processed.csv")
cpi_df = pd.read_csv("../data/cpi_processed.csv")
pmi_df = pd.read_csv("../data/pmi_processed.csv")

plot_gdp(gdp_df)
plot_cpi(cpi_df)
plot_pmi(pmi_df)

print("图表生成完成。")

findfont: Generic family 'sans-serif' not found because none of the following families were found: WenQuanYi Zen Hei
findfont: Generic family 'sans-serif' not found because none of the following families were found: WenQuanYi Zen Hei
findfont: Generic family 'sans-serif' not found because none of the following families were found: WenQuanYi Zen Hei
findfont: Generic family 'sans-serif' not found because none of the following families were found: WenQuanYi Zen Hei
findfont: Generic family 'sans-serif' not found because none of the following families were found: WenQuanYi Zen Hei
findfont: Generic family 'sans-serif' not found because none of the following families were found: WenQuanYi Zen Hei
findfont: Generic family 'sans-serif' not found because none of the following families were found: WenQuanYi Zen Hei
/var/folders/65/_j943sr51sj57lpkjhk8mwkw0000gn/T/ipykernel_38947/3019396724.py:17: UserWarning: Glyph 65288 (\N{FULLWIDTH LEFT PARENTHESIS}) missing from font(s) DejaVu Sans.
  plt.

图表生成完成。


## 4. 经济周期分析与策略建议

基于GDP、CPI和PMI的最新数据，我们应用经济周期理论，判断当前中国经济所处的阶段，并针对国家、企业、投资者和风控层面给出相应的策略建议。

In [ ]:
import pandas as pd

def analyze_economic_cycle(gdp_df, cpi_df, pmi_df):
    latest_gdp_growth = gdp_df["GDP_亿元_同比"].iloc[-1]
    latest_cpi = cpi_df["CPI_同比"].iloc[-1]
    latest_pmi = pmi_df["PMI"].iloc[-1]

    economic_stage = "未知"
    recommendations = {
        "国家层面": "",
        "企业层面": "",
        "投资者层面": "",
        "风控层面": ""
    }

    if latest_gdp_growth > 0 and latest_cpi > 0 and latest_pmi > 50:
        economic_stage = "扩张阶段"
        recommendations["国家层面"] = "继续实施积极的财政政策和稳健的货币政策，关注通胀压力，适时进行微调。"
        recommendations["企业层面"] = "扩大产能，增加投资，抓住市场机遇。"
        recommendations["投资者层面"] = "积极配置股票等风险资产，关注成长性行业。"
        recommendations["风控层面"] = "保持适度杠杆，关注市场过热风险。"
    elif latest_gdp_growth > 5 and latest_cpi > 3 and latest_pmi > 55:
        economic_stage = "繁荣阶段"
        recommendations["国家层面"] = "适度收紧货币政策，控制通胀，防止经济过热。"
        recommendations["企业层面"] = "优化产能结构，提高效率，警惕过度扩张。"
        recommendations["投资者层面"] = "逐步降低风险资产配置，关注价值股和防御性资产。"
        recommendations["风控层面"] = "严格控制杠杆，防范资产泡沫和信用风险。"
    elif latest_gdp_growth < 0 or (latest_gdp_growth < 2 and latest_pmi < 50):
        economic_stage = "衰退阶段"
        recommendations["国家层面"] = "实施宽松的财政政策和货币政策，刺激经济增长，稳定就业。"
        recommendations["企业层面"] = "削减成本，优化现金流，保持核心竞争力。"
        recommendations["投资者层面"] = "规避风险资产，关注债券、黄金等避险资产。"
        recommendations["风控层面"] = "加强风险管理，关注企业违约风险。"
    elif latest_gdp_growth < -2 and latest_cpi < -1 and latest_pmi < 45:
        economic_stage = "萧条阶段"
        recommendations["国家层面"] = "加大财政刺激力度，推出大规模基建项目，实行超宽松货币政策。"
        recommendations["企业层面"] = "保存实力，等待复苏，寻找新的增长点。"
        recommendations["投资者层面"] = "持有现金，等待市场底部，分批建仓。"
        recommendations["风控层面"] = "全面收缩，严控风险，避免破产。"
    elif latest_gdp_growth > 0 and latest_cpi < 1 and latest_pmi > 48 and latest_pmi < 52:
        economic_stage = "复苏阶段"
        recommendations["国家层面"] = "继续保持宽松政策，但需关注经济结构调整和新增长动能培育。"
        recommendations["企业层面"] = "逐步恢复生产，谨慎扩张，关注市场需求变化。"
        recommendations["投资者层面"] = "逐步增加风险资产配置，关注周期性行业。"
        recommendations["风控层面"] = "适度放松，但仍需警惕潜在风险。"

    return economic_stage, recommendations

gdp_df = pd.read_csv('../data/gdp_yearly_processed.csv')
cpi_df = pd.read_csv('../data/cpi_processed.csv')
pmi_df = pd.read_csv('../data/pmi_processed.csv')

economic_stage, recommendations = analyze_economic_cycle(gdp_df, cpi_df, pmi_df)

with open('../reports/economic_analysis_report.txt', 'w', encoding='utf-8') as f:
    f.write(f"当前中国经济所处阶段：{economic_stage}\n")
    f.write("策略建议：\n")
    for level, advice in recommendations.items():
        f.write(f"- {level}：{advice}\n")

print("经济分析报告生成完成：economic_analysis_report.txt")

经济分析报告生成完成：economic_analysis_report.txt


## 5. 结论与展望

本报告通过对中国宏观经济数据的分析，为各方提供了当前经济形势的判断和相应的策略建议。未来，我们将继续关注经济动态，及时更新分析报告。